### Задание4
Осуществить поиск концептов аналогично прошлому дз, но с использование классических методов (стэмминг, лемматизация, токенизация) и предобученных языковых моделей.

In [ ]:
import pandas as pd
from google.colab import drive

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import numpy as np

#### Загрузим данные

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
to_find_df = pd.read_csv('drive/MyDrive/to_find.tsv', sep='|')
concepts_df = pd.read_csv('drive/MyDrive/концепты.tsv', sep='\t')

In [ ]:
to_find_df.head()

,id,Text
0,22,The vertigo is rendering me unable to function...
1,23,I am seeing a specialist for the vertigo and u...
2,24,My vertigo has not responded to conservative t...
3,25,"Muscle spasms, muscle twitching, muscle sorene..."
4,26,I took Lexapro for 3 days in 10mg doses and ha...


In [ ]:
concepts_df.head()

,CUI,CONCEPT,SNOMED_CODE
0,C0000765,"Excessive body weight gain,Excessive weight gain",224994002
1,C0701811,"Poor short-term memory,Poor short-term memory",247592009
2,C0002170,"Alopecia,Loss of hair",278040002
3,C0549622,"Sexual Dysfunction,Sexual disorder",231532002
4,C0027497,"Nausea,Nausea",422587007


#### Предобработка Tfidf

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Будем использовать следующие иструменты для предобработки текста
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [ ]:
#Предобработка текста
def preprocess_text(text):
    # Токенизация текста
    tokens = word_tokenize(text.lower())
    # Удаление стоп-слов и стемминг
    tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens if token.isalnum() and token not in stop_words]
    return ' '.join(tokens)


In [ ]:
# Применение предобработки к данным
to_find_df['processed_text'] = to_find_df['Text'].apply(preprocess_text)
concepts_df['processed_concept'] = concepts_df['CONCEPT'].apply(preprocess_text)


In [ ]:
to_find_df.head()

,id,Text,processed_text
0,22,The vertigo is rendering me unable to function...,vertigo render unabl function daili life caus ...
1,23,I am seeing a specialist for the vertigo and u...,see specialist vertigo undergo test diagnos ve...
2,24,My vertigo has not responded to conservative t...,vertigo respond conserv treatment thu far
3,25,"Muscle spasms, muscle twitching, muscle sorene...",muscl spasm muscl twitch muscl sore insomnia m...
4,26,I took Lexapro for 3 days in 10mg doses and ha...,took lexapro 3 day 10mg do sever side effect i...


In [ ]:
concepts_df.head()

,CUI,CONCEPT,SNOMED_CODE,processed_concept
0,C0000765,"Excessive body weight gain,Excessive weight gain",224994002,excess bodi weight gain excess weight gain
1,C0701811,"Poor short-term memory,Poor short-term memory",247592009,poor memori poor memori
2,C0002170,"Alopecia,Loss of hair",278040002,alopecia loss hair
3,C0549622,"Sexual Dysfunction,Sexual disorder",231532002,sexual dysfunct sexual disord
4,C0027497,"Nausea,Nausea",422587007,nausea nausea


#### Поиск совпадений с использованием предобработки текста TfidfVectorizer

In [ ]:
# Векторизация текста
vectorizer = TfidfVectorizer()
text_vectors = vectorizer.fit_transform(to_find_df['processed_text'])
concept_vectors = vectorizer.transform(concepts_df['processed_concept'])


In [ ]:
# Вычисление косинусного сходства
similarity_matrix = cosine_similarity(text_vectors, concept_vectors)


In [ ]:
# Количество топовых совпадений
num_top_matches = 5

In [ ]:
# Поиск топовых совпадений
tops = (-similarity_matrix).argsort()[:, :num_top_matches]

In [ ]:
start_time = time.time()
result = pd.concat(
    [
        pd.concat(
            [
                to_find_df.loc[[i], ['Text', 'id']].reset_index(drop=True),
                concepts_df.loc[[t], ['CUI', 'CONCEPT']].reset_index(drop=True),
                pd.Series(similarity_matrix[i][t], name='Similarity')
            ],
            axis=1
        )
        for i in range(len(to_find_df)) for t in tops[i]
    ]
).reset_index(drop=True)

elapsed_time = (time.time() - start_time) / 60

result.head(), elapsed_time

(                                                Text  id        CUI  \
 0  The vertigo is rendering me unable to function...  22  C0036104    
 1  The vertigo is rendering me unable to function...  22  C0557386    
 2  The vertigo is rendering me unable to function...  22  C0413862    
 3  The vertigo is rendering me unable to function...  22  C1283330    
 4  The vertigo is rendering me unable to function...  22   C0557386   
 
                                              CONCEPT  Similarity  
 0   secretion of saliva,Salivary secretion, function     0.34457  
 1  Restricted work performance,Restricted work pe...     0.34457  
 2  Adverse reaction to caffeine,Adverse reaction ...     0.34457  
 3            Absenteeism at work,Absenteeism at work     0.34457  
 4  Restricted work performance,Restricted work pe...     0.34457  ,
 0.004673186937967936)

In [ ]:
#Посмотрим результаты и уверенность в них
result

,Text,id,CUI,CONCEPT,Similarity
0,The vertigo is rendering me unable to function...,22,C0036104,"secretion of saliva,Salivary secretion, function",0.344570
1,The vertigo is rendering me unable to function...,22,C0557386,"Restricted work performance,Restricted work pe...",0.344570
2,The vertigo is rendering me unable to function...,22,C0413862,"Adverse reaction to caffeine,Adverse reaction ...",0.344570
3,The vertigo is rendering me unable to function...,22,C1283330,"Absenteeism at work,Absenteeism at work",0.344570
4,The vertigo is rendering me unable to function...,22,C0557386,"Restricted work performance,Restricted work pe...",0.344570
...,...,...,...,...,...
110,I started feeling the side effects the fist da...,44,C0583786,"Seen in psychiatry day hospital,Seen in psychi...",0.388010
111,I started feeling the side effects the fist da...,44,C0085631,"Agitation,Feeling agitated",0.351762
112,I started feeling the side effects the fist da...,44,C0522165,"Feeling calm,Feeling calm",0.351762
113,I started feeling the side effects the fist da...,44,C0557883,"Feeling aggressive,Feeling aggressive",0.351762


#### Поиск совпадений с использованием предобработки текста word2vec

In [ ]:
# Предобработка текста с помощью word2vec
def preprocess_for_word2vec(text):
    return simple_preprocess(text, deacc=True)

In [ ]:
# Применение предобработки
to_find_df['processed_tokens'] = to_find_df['Text'].apply(preprocess_for_word2vec)
concepts_df['processed_tokens'] = concepts_df['CONCEPT'].apply(preprocess_for_word2vec)


In [ ]:
# Объединение данных для обучения модели Word2Vec
all_texts = to_find_df['processed_tokens'].tolist() + concepts_df['processed_tokens'].tolist()


In [ ]:
# Обучение модели Word2Vec
word2vec_model = Word2Vec(sentences=all_texts, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
# Функция для получения среднего вектора текста
def get_average_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [ ]:
# Преобразование текстов в векторные представления
to_find_df['vector'] = to_find_df['processed_tokens'].apply(lambda tokens: get_average_vector(tokens, word2vec_model))
concepts_df['vector'] = concepts_df['processed_tokens'].apply(lambda tokens: get_average_vector(tokens, word2vec_model))


In [ ]:
# Вычисление косинусного сходства
similarity_matrix = cosine_similarity(np.vstack(to_find_df['vector']), np.vstack(concepts_df['vector']))


In [ ]:
# Количество топовых совпадений
num_top_matches = 5

In [ ]:
# Поиск топовых совпадений
tops = (-similarity_matrix).argsort()[:, :num_top_matches]

In [ ]:
# Создание таблицы с результатами
result = pd.concat(
    [
        pd.concat(
            [
                to_find_df.loc[[i], ['Text', 'id']].reset_index(drop=True),
                concepts_df.loc[[t], ['CUI', 'CONCEPT']].reset_index(drop=True),
                pd.Series(similarity_matrix[i][t], name='Similarity')
            ],
            axis=1
        )
        for i in range(len(to_find_df)) for t in tops[i]
    ]
).reset_index(drop=True)

In [ ]:
#Посмотрим результаты и уверенность в них
result

,Text,id,CUI,CONCEPT,Similarity
0,The vertigo is rendering me unable to function...,22,C4048182,"Unable to swallow,Unable to swallow",0.623002
1,The vertigo is rendering me unable to function...,22,C0564216,"Unable to speak,Unable to speak",0.572340
2,The vertigo is rendering me unable to function...,22,C0235198,"Unable to concentrate,Unable to concentrate",0.563954
3,The vertigo is rendering me unable to function...,22,C0582140,"Unable to control behavior,Unable to control b...",0.557098
4,The vertigo is rendering me unable to function...,22,C4074971,"Unable to cry,Unable to cry",0.542097
...,...,...,...,...,...
110,I started feeling the side effects the fist da...,44,C0522165,"Feeling calm,Feeling calm",0.378761
111,I started feeling the side effects the fist da...,44,C0424000,"Feeling suicidal,Feeling suicidal",0.341450
112,I started feeling the side effects the fist da...,44,C0849963,"Feeling nervous,Feeling nervous",0.308955
113,I started feeling the side effects the fist da...,44,C0557885,"Feeling trapped,Feeling trapped",0.288008


#### Поиск совпадений с использованием предобработки текста BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def get_bert_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

In [ ]:
# Применение BERT для получения эмбеддингов текстов
to_find_df['bert_embeddings'] = to_find_df['Text'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))
concepts_df['bert_embeddings'] = concepts_df['CONCEPT'].apply(lambda x: get_bert_embeddings(x, tokenizer, model))


In [ ]:
# Вычисление косинусного сходства между эмбеддингами
similarity_matrix = cosine_similarity(
    np.vstack(to_find_df['bert_embeddings']),
    np.vstack(concepts_df['bert_embeddings'])
)

In [ ]:
# Количество топовых совпадений
num_top_matches = 5

In [ ]:
# Поиск топовых совпадений
tops = (-similarity_matrix).argsort()[:, :num_top_matches]


In [ ]:
# Создание таблицы с результатами
result = pd.concat(
    [
        pd.concat(
            [
                to_find_df.loc[[i], ['Text', 'id']].reset_index(drop=True),
                concepts_df.loc[[t], ['CUI', 'CONCEPT']].reset_index(drop=True),
                pd.Series(similarity_matrix[i][t], name='Similarity')
            ],
            axis=1
        )
        for i in range(len(to_find_df)) for t in tops[i]
    ]
).reset_index(drop=True)

In [ ]:
#Посмотрим результаты и уверенность в них
result

,Text,id,CUI,CONCEPT,Similarity
0,The vertigo is rendering me unable to function...,22,C0557885,"Feeling trapped,Feeling trapped",0.870501
1,The vertigo is rendering me unable to function...,22,C0455204,"Homicidal thoughts,Homicidal thoughts",0.868283
2,The vertigo is rendering me unable to function...,22,C0701812,"Sluggish thinking,Sluggish thought",0.865971
3,The vertigo is rendering me unable to function...,22,C0586741,"Unable to read,Unable to read",0.864472
4,The vertigo is rendering me unable to function...,22,C0235198,"Unable to concentrate,Unable to concentrate",0.864060
...,...,...,...,...,...
110,I started feeling the side effects the fist da...,44,C0024031,"Low Back Pain,Low back pain",0.783334
111,I started feeling the side effects the fist da...,44,C0016382,"Flushing,Face goes red",0.779499
112,I started feeling the side effects the fist da...,44,C0033893,"Tension Headache,Tension-type headache",0.779284
113,I started feeling the side effects the fist da...,44,C0262630,"Reduced concentration span,Reduced concentrati...",0.777194


Наилучшие результаты для оценки сходства между текстами показало использование в прдобработке данных BERT (Bidirectional Encoder Representations from Transformers), так как эта модель способна учитывать контекст и сложные семантические отношения благодаря своей двунаправленной архитектуре, в отличие от моделей, основанных на tfidf и word2vec. 

In [77]:
#Запишем лучшие результаты в файл:
result.to_csv('BERT_results.csv', index=False, encoding='utf-8')